# Минипроект Бронирование отелей

Имеются следующие переменные:

- Hotel – тип отеля (City Hotel или Resort Hotel)  
- Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным
- Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
- Arrival full date – полная дата прибытия
- Arrival date year – год прибытия  
- Arrival date month – месяц прибытия  
- Arrival date week number – номер недели прибытия
- Arrival date day of month – день прибытия
- Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
- Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
- Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
- Adults – число взрослых
- Children – число детей
- Babies – число младенцев 
- Meal – выбранный тип питания
- Country – страна происхождения клиента
- Reserved room type – тип зарезервированного номера
- Assigned room type – тип полученного номера (может отличаться от забронированного)
- Customer type – тип бронирования
- Reservation status – значение последнего статуса брони: Canceled - было отменено клиентом; Check-Out - клиент зарегистрировался, но уже покинул отель; No-Show - клиент не зарегистрировался и сообщил администрации отеля причину
- Reservation status date – дата обновления статуса

Задачи:
- загрузить и изучить датасет
- решить несколько аналитических задач

## Загрузка и изучение датасета

In [1]:
# импорт библиотеки
import pandas as pd

In [2]:
# загрузка датасета с разделителем ;
bookings = pd.read_csv('bookings.csv', sep=";")

In [3]:
# первые 7 строк датасета
bookings_head = bookings.head(7)

In [4]:
bookings_head

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [5]:
# размер датасета
bookings.shape

(119390, 21)

In [6]:
# типы данных в датасете
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [7]:
# названия колонок в датасете
bookings.columns

Index(['Hotel', 'Is Canceled', 'Lead Time', 'arrival full date',
       'Arrival Date Year', 'Arrival Date Month', 'Arrival Date Week Number',
       'Arrival Date Day of Month', 'Stays in Weekend nights',
       'Stays in week nights', 'stays total nights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'Reserved Room Type', 'Assigned room type',
       'customer type', 'Reservation Status', 'Reservation status_date'],
      dtype='object')

In [8]:
# переименуем колонки в стиле snake_case
def replace_name(name):
    new_name = name.replace(' ', '_').lower()
    return new_name

In [9]:
bookings = bookings.rename(columns=replace_name)

In [10]:
# проверим изменения
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

## Исследование данных

1. Пользователи из каких стран совершили наибольшее число успешных бронирований? Бронирование считается успешным, если в дальнейшем не было отменено (переменная is_canceled). Выбрать топ-5 стран.

In [11]:
bookings.query('is_canceled == 0')\
    .groupby('country', as_index=False)\
    .agg({'is_canceled': 'count'})\
    .sort_values('is_canceled', ascending=False)\
    .head()

,country,is_canceled
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


2. На сколько ночей (stays_total_nights)  в среднем бронируют отели типа City Hotel? Resort Hotel? Округлить значения до 2-х знаков после запятой.

In [12]:
bookings.groupby('hotel', as_index=False).agg({'stays_total_nights': 'mean'}).round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


3. Иногда тип номера, присвоенного клиенту (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?

In [13]:
bookings.query('assigned_room_type != reserved_room_type').shape[0]

14917

4. Теперь проанализируйте даты запланированного прибытия (arrival_date_year). На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017?

In [14]:
bookings.query('arrival_date_year == 2016').arrival_date_month.value_counts().sort_values(ascending=False)

October      6203
May          5478
April        5428
September    5394
June         5292
August       5063
March        4824
July         4572
November     4454
February     3891
December     3860
January      2248
Name: arrival_date_month, dtype: int64

In [15]:
bookings.query('arrival_date_year == 2017').arrival_date_month.value_counts().sort_values(ascending=False)

May         6313
April       5661
June        5647
July        5313
March       4970
August      4925
February    4177
January     3681
Name: arrival_date_month, dtype: int64

5. Сгруппируйте данные по годам, а затем проверьте, на какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 

In [22]:
bookings.query('hotel == "City Hotel" and is_canceled == 1')\
    .groupby('arrival_date_year')['arrival_date_month']\
    .value_counts()

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

6. Посмотрите на числовые характеристики трёх колонок: adults, children и babies. Какая из них имеет наибольшее среднее значение?

In [27]:
bookings[['adults', 'children', 'babies']].mean().idxmax()

'adults'

7. Создайте колонку total_kids, объединив столбцы children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим? Значение округлить до 2-х знаков после запятой.

In [28]:
# создание новой колонки
bookings['total_kids'] = bookings.children + bookings.babies

In [38]:
bookings.groupby('hotel')['total_kids'].mean().round(2)

hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

8. Не все бронирования завершились успешно (is_canceled), поэтому можно посчитать, сколько клиентов было потеряно в процессе. Иными словами, посчитать метрику под названием Churn Rate. Посмотрим, как эта метрика связана с наличием детей у клиентов.
Создайте колонку has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Далее проверьте, среди какой группы пользователей показатель оттока выше. 

In [39]:
# напишем функцию для создания новой колонки: если есть хоть один ребенок - значение True
def has_kids(df):
    if df['total_kids'] >= 1:
        value = True
    else:
        value = False
    return value

In [41]:
# создадим новую колонку
bookings['has_kids'] = bookings.apply(has_kids, axis=1)

In [50]:
bookings['has_kids'].value_counts(normalize=True)

False    0.921836
True     0.078164
Name: has_kids, dtype: float64

In [48]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids,has_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0,False
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0,False
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0,False


In [54]:
# клиенты с детьми и отменившие бронь
has_kids_canceled = bookings.query('has_kids == 1 and is_canceled == 1').shape[0]

In [55]:
# все клиенты с детьми
has_kids_all = bookings.query('has_kids == 1').shape[0]

In [61]:
# коэффициент оттока клиентов с детьми, округленный до 2-х знаков
churn_kids = round(((has_kids_canceled / has_kids_all) * 100), 2)

In [62]:
churn_kids

34.92

In [63]:
# клиенты без детей и отменившие бронь
no_kids_canceled = bookings.query('has_kids == 0 and is_canceled == 1').shape[0]

In [64]:
# все клиенты без детей
no_kids_all = bookings.query('has_kids == 0').shape[0]

In [65]:
# коэффициент оттока клиентов без детей, округленный до 2-х знаков
churn_no_kids = round(((no_kids_canceled / no_kids_all) * 100), 2)

In [66]:
churn_no_kids

37.22

Коэффициент отмены бронирований у клиентов без детей намного выше, чем у клиентов с детьми (34.92% против 37.22%).

## Выводы

В ходе данного минипроекта было сделано:
- загружен и изучен датасет
- написана функция для изменения названий колонок в стиле snake_case
- вычислены топ-5 стран с наибольшим кол-вом бронирований
- вычислено среднее кол-во забронированных ночей в разных типах отелей
- вычислено кол-во изменений типов номеров при бронировании и во время заселения
- выявлены самые популярные месяцы для бронирования
- найдены месяца с самым большим кол-вом отмен бронирований в разные годы
- вычислены средние значения для категорий клиентов: взрослые, дети, младенцы
- создан новый столбец с общим кол-вом детей и выявлен тип отеля, где чаще отдыхают клиенты с детьми
- написана функция для создания нового столбца, показывающего наличие детей у клиента, и посчитан коэффициент отмены бронирований у клиентов разных групп по наличию детей